### Importaciones de Bibliotecas

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import pylab as plt
import seaborn as sns

### Carga de los datos

In [2]:
salaries = pd.read_csv('data/salaries_data.csv')
testeo = pd.read_csv('data/testeo.csv')

In [3]:
salarios_usd = salaries['salary_in_usd']

In [4]:
df = salaries.drop(['salary','salary_currency','salary_in_usd'], axis=1)

In [5]:
total = pd.concat([df,testeo], axis=0) # Concatenamos los dos dataframes para que tengan los mismos cambios en los datos

## Proceso de Limpieza y transformacion

In [6]:
from sklearn.preprocessing import OrdinalEncoder

# Creamos el codificador indicandole el orden de la variables
encoder = OrdinalEncoder(categories=[['EN','MI','SE','EX']])

# Ajustamos el codificador con la variable experience_level y la transformamos
encoder.fit(total[["experience_level"]])
total["experience_level_encoded"] = encoder.transform(total[["experience_level"]])

In [7]:
total.drop(['experience_level'], axis=1, inplace=True)

In [8]:
# Creamos el codificador indicandole el orden de la variables
encoder = OrdinalEncoder(categories=[['PT','FL','FT','CT']])

# Ajustamos el codificador con la variable experience_level y la transformamos
encoder.fit(total[["employment_type"]])
total["employment_type_encoded"] = encoder.transform(total[["employment_type"]])
total.drop(['employment_type'], axis=1, inplace=True)

In [9]:
total = pd.get_dummies(total, columns=['company_size'])

In [42]:
dictio = {'US': 20934.6 , 'FR': 2582.5,  'GR': 185.5 ,  'LU': 70.9,  'SI': 57.3 , 'DE': 3847.2,  'IN': 2689.9 ,'GB': 2622.4 , 'PK' : 263.9,  'MD' : 4.3,  'JP' : 4872.4, 'CA': 1545.4, 'AS': 0.8,  'IE': 383.4, 'AE': 421.1, 'MX': 1046.6,  'VN' : 341.2, 'BE': 531.2, 'KE': 97.9, 'ES': 1236.3, 'CH': 703.1, 'CL': 224.6,  'CN': 14720.9 , 'DK': 306.3, 'TR': 717.9, 'NZ': 206.7, 'PL': 595.6, 'UA': 153.1, 'AU': 1358.3, 'NG': 448.1, 'EE': 29.2, 'CZ': 257.9, 'AT': 455.5, 'BR': 1363.6, 'DZ': 146.4, 'IR': 439.5, 'NL': 902.3, 'HU': 166.2, 'PT': 229.0, 'HN': 24.7, 'MT': 16.9, 'RO': 236.5, 'SG': 340.4, 'IT': 1595.8, 'HR': 54.9, 'IQ': 171.0, 'IL': 387.2, 'RU': 1481.4, 'CO': 314.4, 'MY': 336.4, 'JE': 5.569, 'BG': 84.06, 'HK' : 369.2, 'PR':106.5,'RS':63.08, 'BO':40.41, 'PH': 394.1,'AR': 487.2,'TN':46.69 }


In [11]:
total.company_location = total.company_location.apply(lambda x: dictio[x])

In [43]:
total.employee_residence = total.employee_residence.apply(lambda x: dictio[x])

In [44]:
def agrupar_titulos(titulo):
    if 'data engineer' in titulo.lower():
        return 'Data Engineer'
    elif 'data scientist' in titulo.lower():
        return 'Data Scientist'
    elif 'data analyst' in titulo.lower():
        return 'Data Analyst'
    elif 'machine learning' or 'ml' in titulo.lower():
        return 'Machine Learning'
    else:
        return titulo

In [45]:
total.job_title = total.job_title.apply(agrupar_titulos)

In [46]:
total = pd.get_dummies(total, columns=['job_title'])
total.head()

,work_year,employee_residence,remote_ratio,company_location,experience_level_encoded,employment_type_encoded,company_size_L,company_size_M,company_size_S,job_title_Data Analyst,job_title_Data Engineer,job_title_Data Scientist,job_title_Machine Learning
0,2022,20934.6,100,20934.6,2.0,2.0,0,1,0,0,1,0,0
1,2022,20934.6,100,20934.6,2.0,2.0,0,1,0,0,1,0,0
2,2021,20934.6,100,20934.6,1.0,2.0,0,1,0,1,0,0,0
3,2021,20934.6,100,20934.6,1.0,3.0,1,0,0,0,0,0,1
4,2021,236.5,0,20934.6,1.0,2.0,1,0,0,0,1,0,0


In [47]:
prueba = total.iloc[:500]
testeo = total.iloc[500:]

In [48]:
prueba = prueba.join(salarios_usd)  # uniendo el DataFrame df2 al DataFrame df1

## Separacion de variables para entrenar el modelo

In [49]:
X = prueba

y = salarios_usd 

In [51]:
# train_test_split

from sklearn.model_selection import train_test_split as tts  # el alias es cosa mia

X_train, X_test, y_train, y_test  = tts(X, y, train_size=0.8, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((400, 14), (100, 14), (400,), (100,))

### H20 como modelo de ML para entrenar y testear

In [52]:
import h2o

from h2o.automl import H2OAutoML

In [53]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.361-b09, mixed mode)
  Starting server from C:\Users\Carolina\AppData\Local\Programs\Python\Python310\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Carolina\AppData\Local\Temp\tmpbmx3w7y6
  JVM stdout: C:\Users\Carolina\AppData\Local\Temp\tmpbmx3w7y6\h2o_Carolina_started_from_python.out
  JVM stderr: C:\Users\Carolina\AppData\Local\Temp\tmpbmx3w7y6\h2o_Carolina_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,10 secs
H2O_cluster_timezone:,Europe/Paris
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.1
H2O_cluster_version_age:,26 days
H2O_cluster_name:,H2O_from_python_Carolina_obopkr
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,420.9 Mb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [54]:
h2train=h2o.H2OFrame(prueba)


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [55]:
h2test=h2o.H2OFrame(testeo)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [56]:
X=[c for c in h2train.columns if c!='salary_in_usd']
y = 'salary_in_usd'

In [57]:
# inicia auto-machine-learning

automl=H2OAutoML(max_models=20,
                 seed=42,   # random_state
                 max_runtime_secs=300,
                 sort_metric='RMSE')


In [58]:
# entrena

automl.train(x=X,
             y=y,
             training_frame=h2train)

AutoML progress: |
11:09:10.370: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_2_AutoML_1_20230307_110910


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    40                 40                          13794                  7            7            7             19            27            22.725

ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 1629274314.7104878
RMSE: 40364.27027347934
MAE: 27028.435369140625
RMSLE: 0.4004582930937642
Mean Residual Deviance: 1629274314.7104878

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 2358448233.792908
RMSE: 48563.85727877171
MAE: 33059.064579983526
RMSLE: 0.5149484720051886
Mean Residual Deviance: 2358448233.792908

Cross-Validation Metrics Summary: 
                        mean         sd         cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
----------------------  -----------  ---------  ------------  ------------  ------------  ------------  ------------
mae                     33259.6      3430.73    35879.1       31026.6       28762.8       33503.2       37126.3
mean_residual_deviance  2.37894e+09  8.57e+08   3.40519e+09   1.56009e+09   1.50893e+09   2.36554e+09   3.05495e+09
mse                     2.37894e+09  8.57e+08   3.40519e+09   1.56009e+09   1.50893e+09   2.36554e+09   3.05495e+09
r2                      0.512667     0.0943661  0.405954      0.629164      0.532658      0.567844      0.427718
residual_deviance       2.37894e+09  8.57e+08   3.40519e+09   1.56009e+09   1.50893e+09   2.36554e+09   3.05495e+09
rmse                    48121.1      8895.41    58354         39498         38844.9       48636.8       55271.6
rmsle                   0.504869     0.0705851  0.42537       0.590269      0.445409      0.505702      0.557592

Scoring History: 
    timestamp            duration    number_of_trees    training_rmse    training_mae    training_deviance
--  -------------------  ----------  -----------------  ---------------  --------------  -------------------
    2023-03-07 11:09:20  0.958 sec   0                  69544.1          52000.6         4.83638e+09
    2023-03-07 11:09:20  0.978 sec   5                  55523.8          39483.6         3.08289e+09
    2023-03-07 11:09:20  1.003 sec   10                 48592.2          33445.8         2.36121e+09
    2023-03-07 11:09:20  1.027 sec   15                 45331.6          31020           2.05495e+09
    2023-03-07 11:09:20  1.045 sec   20                 43485.1          29491.9         1.89096e+09
    2023-03-07 11:09:20  1.061 sec   25                 42403.9          28660.3         1.79809e+09
    2023-03-07 11:09:20  1.082 sec   30                 41455            27928.7         1.71852e+09
    2023-03-07 11:09:20  1.100 sec   35                 40801.8          27458.3         1.66479e+09
    2023-03-07 11:09:20  1.119 sec   40                 40364.3          27028.4         1.62927e+09

Variable Importances: 
variable                    relative_importance    scaled_importance    percentage
--------------------------  ---------------------  -------------------  ------------
employee_residence          3.18985e+12            1                    0.382049
company_location            1.63095e+12            0.511292             0.195339
experience_level_encoded    1.44475e+12            0.452921             0.173038
job_title_Machine Learning  3.56636e+11            0.111803             0.0427143
company_size_L              3.09436e+11            0.0970064            0.0370612
job_title_Data Analyst      3.06839e+11            0.0961922            0.0367501
remote_ratio                3.05856e+11            0.0958841            0.0366324
work_year   

In [59]:
# prediciones del lider

y_pred=automl.leader.predict(h2test)

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [60]:
muestra = pd.read_csv('data/muestra.csv')
muestra

,id,salary_in_usd
0,0,26352
1,1,40574
2,2,68759
3,3,74733
4,4,32611
...,...,...
102,102,83373
103,103,57827
104,104,31053
105,105,39725


### Subir los resultados a un .CSV

In [61]:
pred_df = y_pred.as_data_frame()

In [62]:
muestra['salary_in_usd'] = pred_df 

In [63]:
muestra.to_csv('data/hsegundointento.csv', index=False)